In [20]:
import pandas as pd 
from statistics import mean
import time 
import re

In [15]:
#Input path
path_tablajose = 'C:/Users/paula/OneDrive/Documentos/0. Master Biologia Computacional UPM/TFM_CNB/Automatizated process/Tables/DatabaseSamplesJOSE.csv'
path_tablasra = 'C:/Users/paula/OneDrive/Documentos/0. Master Biologia Computacional UPM/TFM_CNB/Automatizated process/Tables/SraRunInfo.csv'

path_tablaweb = 'C:/Users/paula/OneDrive/Documentos/0. Master Biologia Computacional UPM/TFM_CNB/Automatizated process/Tables/SRAinformation.tsv'
path_infotable = 'C:/Users/paula/OneDrive/Documentos/0. Master Biologia Computacional UPM/TFM_CNB/Automatizated process/Tables/sra_result.csv'

#Output path
path_output_tables = 'C:/Users/paula/OneDrive/Documentos/0. Master Biologia Computacional UPM/TFM_CNB/Automatizated process/Tables/'

In [16]:
#Upload files
dfjose = pd.read_csv(path_tablajose)
dfsra = pd.read_csv(path_tablasra)

dfweb = pd.read_csv(path_tablaweb,sep='\t')
dfinfo = pd.read_csv(path_infotable)

C:\Users\paula\AppData\Local\Temp\ipykernel_3060\2166315934.py:3: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  dfsra = pd.read_csv(path_tablasra)


In [23]:
##MERGED BY SRX
#Intersección: los que están en ambos.
dfmerge = pd.merge(dfjose, dfsra, 
                   on=['Experiment'], 
                   how='inner')

# dfmerge.to_csv(f'{path_output_tables}Merge.csv', index=False)

#Tabla de Jose sin los comunes.
comun = list(dfmerge['Experiment'])
indexNames = dfjose[dfjose['Experiment'].isin(comun)].index
dfjosedrop = dfjose.drop(indexNames)

# dfjosedrop.to_csv(f'{path_output_tables}Jose(-Merge).csv', index=False)

#Tabla de SRA sin los comunes.
indexNames1 = dfsra[dfsra['Experiment'].isin(comun)].index
dfsradrop = dfsra.drop(indexNames1)

# dfsradrop.to_csv(f'{path_output_tables}Sra(-Merge).csv', index=False)


##MERGED BY GSM
dfjosedrop.rename(columns = {'Experiment':'geo'}, inplace = True)
dfsradrop.rename(columns = {'SampleName':'geo'}, inplace = True)

dfmerge_geo = pd.merge(dfjosedrop, dfsradrop, 
                   on=['geo'], 
                   how='inner')
#-------------------------------------------------------------------------------------------------------------------------------

##CONCACT TWO MERGED
dfmerge_geo.rename(columns = {'geo':'SampleName_y'}, inplace = True)
dfmerge_geo.rename(columns = {'SampleName':'SampleName_x'}, inplace = True)

merged_final = pd.concat([dfmerge, dfmerge_geo])
# merge_final.to_csv(f'{path_output_tables}Merge_final.csv', index=False)
#-------------------------------------------------------------------------------------------------------------------------------

##SRA without the merged_final runs
comunruns = list(merge_final['Run'])
indexNames2 = dfsra[dfsra['Run'].isin(comunruns)].index
dfsra_without = dfsra.drop(indexNames2)

# dfsra_without.to_csv(f'{path_output_tables}Sra(-Merge_final).csv', index=False)

##SRA(-Merge_final) Filter by columns values:
dfsra_without = dfsra_without.drop(dfsra_without[(dfsra_without.LibrarySelection == 'MNase')|(dfsra_without.LibrarySelection == 'Restriction Digest')|(dfsra_without.LibrarySelection == 'padlock probes capture metho')].index)
dfsra_without = dfsra_without.drop(dfsra_without[(dfsra_without.LibrarySource == 'GENOMIC')|(dfsra_without.LibrarySource == 'METAGENOMIC')|(dfsra_without.LibrarySource == 'TRANSCRIPTOMIC SINGLE CELL')|(dfsra_without.LibrarySource == 'METATRANSCRIPTOMIC')|(dfsra_without.LibrarySource == 'SYNTHETIC')|(dfsra_without.LibrarySource == 'VIRAL RNA')].index)
dfsra_without = dfsra_without.drop(dfsra_without[(dfsra_without.Platform == 'LS454')|(dfsra_without.Platform == 'OXFORD_NANOPORE')|(dfsra_without.Platform == 'HELICOS')|(dfsra_without.Platform == 'PACBIO_SMRT')].index)
dfsra_without = dfsra_without.drop(dfsra_without[(dfsra_without.Model == 'PacBio RS')|(dfsra_without.Model == 'Helicos HeliScope')|(dfsra_without.Model == 'Illumina HiSeq X')|(dfsra_without.Model == 'HiSeq X Five')|(dfsra_without.Model == 'Illumina iSeq 100')|(dfsra_without.Model == 'NextSeq 2000')|(dfsra_without.Model == 'DNBSEQ-T7')|(dfsra_without.Model == 'GridION')|(dfsra_without.Model == 'PromethION')|(dfsra_without.Model == 'Sequel II')|(dfsra_without.Model == 'NextSeq 1000')|(dfsra_without.Model == 'DNBSEQ-G400')|(dfsra_without.Model == '454 GS FLX+')|(dfsra_without.Model == 'MGISEQ-2000RS')|(dfsra_without.Model == 'PacBio RS II')|(dfsra_without.Model == 'Sequel')|(dfsra_without.Model == '454 GS')|(dfsra_without.Model == '454 GS FLX Titanium')|(dfsra_without.Model == 'MinION')].index)

# dfsra_without.to_csv(f'{path_output_tables}Sra(-Merge_final)Filtered.csv', index=False)
#-------------------------------------------------------------------------------------------------------------------------------

##Rename Jose columns in merged_final and cleaning

merged_final.rename(columns = {'Unnamed: 11':'Jose_1'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 12':'Jose_2'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 13':'Jose_3'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 14':'Jose_4'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 15':'Jose_5'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 16':'Jose_6'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 17':'Jose_7'}, inplace = True)
merged_final.rename(columns = {'Unnamed: 18':'Jose_8'}, inplace = True)

merged_final.Jose_1 = merged_final.Jose_1.str.strip()
merged_final.Jose_1 = merged_final.Jose_1.str.replace('\t', '')


merged_final = merged_final.replace('abitotic','abiotic', regex=True)
merged_final = merged_final.replace('Abiotic','abiotic', regex=True)
merged_final = merged_final.replace('developemt','development', regex=True)
merged_final = merged_final.replace('developement','development', regex=True)
merged_final = merged_final.replace('developmet','development', regex=True)
merged_final = merged_final.replace('homone','hormone', regex=True)
merged_final = merged_final.replace('hormome','hormone', regex=True)
merged_final = merged_final.replace('nioinfo','noinfo', regex=True)
merged_final = merged_final.replace('no info','noinfo', regex=True)
merged_final = merged_final.replace('Light','light', regex=True)
merged_final = merged_final.replace('methylayion','methylation', regex=True)
merged_final = merged_final.replace('nutrients','nutrient', regex=True)
merged_final = merged_final.replace('nutirent','nutrient', regex=True)
merged_final = merged_final.replace('chenical','chemical', regex=True)
merged_final = merged_final.replace('chormatin','chromatin', regex=True)
merged_final = merged_final.replace('mutatant/Ox','mutant/Ox', regex=True)
merged_final = merged_final.replace('accessions','Accessions', regex=True)
merged_final = merged_final.replace('Accession','Accessions', regex=True)
merged_final = merged_final.replace('accession','Accessions', regex=True)
merged_final = merged_final.replace('Accessionss','Accessions', regex=True)
merged_final = merged_final.replace('accesions','Accessions', regex=True)

merged_final = merged_final.replace('Accessionss','Accessions', regex=True)

merged_final = merged_final.replace('methylation','chromatin', regex=True)
merged_final = merged_final.replace('nutrient stress','nutrient', regex=True)
merged_final = merged_final.replace('beneficial biotic','biotic', regex=True)
merged_final = merged_final.replace('flower','development', regex=True)

# print(merged_final.Jose_1.value_counts())

# merged_final.to_csv(f'{path_output_tables}Merge_final_GoodClasif.csv', index=False)
# ------------------------------------------------------------------------------------------------------------------------------

##Ordenar tabla merge final good clasification:

dfweb.rename(columns={'Unnamed: 0':'BioProject','0':'Project Tittle','1':'Project Description','2':'Project Biblio'}, inplace = True)
dfinfo.rename(columns={"Experiment Accession": "Experiment"},inplace = True)

dfmerge_final_web = pd.merge(merged_final, dfweb, 
                   on=['BioProject'], 
                   how='inner')
dfmerge_final_webinfo = pd.merge(dfmerge_final_web, dfinfo, 
                   on=['Experiment'], 
                   how='inner')

dfmerge_final_webinfo.drop(['LibraryStrategy','LibrarySelection','LibrarySource'], axis=1,inplace = True)
merge_final_ordenada = dfmerge_final_webinfo.reindex(columns=['Submission','Study Accession','Experiment','Sample Accession','Run',
                                         'BioProject','ProjectID','BioSample','Organism Name','SampleType',
                                         'ReleaseDate_y','LoadDate','CenterName','Submitter',
                                         'SampleName_x','LibraryName','Library Name','SampleName_y','Library Strategy', 'Library Source','Library Selection','LibraryLayout',
                                         'TotalReads','UniqueMappedRatio','bases','spots','spots_with_mates','avgLength','size_MB', 'Total Size, Mb', 'Total RUNs', 'Total Spots','Total Bases',
                                         'InsertSize','InsertDev','Platform','Instrument',
                                         'Study Title','Sample Title','Project Tittle','Project Description','Project Biblio',
                                         'Jose_1', 'Jose_2', 'Jose_3', 'Jose_4', 'Jose_5','Jose_6', 'Jose_7', 'Jose_8',
                                         'download_path'])

merge_final_ordenada.rename(columns={"Sample Title": "SampleTitle"},inplace = True)
merge_final_ordenada.rename(columns={"ReleaseDate_y": "ReleaseDate"},inplace = True)

# merge_final_ordenada.to_csv(f'{path_output_tables}Merge_final_GoodClasif_ordenada.csv', index=False)


##Ordenar tabla SRA(-Merge_final) good clasification:
dfsra_without_web = pd.merge(dfsra_without, dfweb, 
                   on=['BioProject'], 
                   how='inner')
dfsra_without_webinfo = pd.merge(dfsra_without_web, dfinfo, 
                   on=['Experiment'], 
                   how='inner')

sra_without_ordenada = dfsra_without_webinfo.reindex(columns=['Submission','Study Accession','Experiment','Sample','Run',
                                         'BioProject','ProjectID','BioSample','Organism Name','SampleType',
                                         'ReleaseDate','LoadDate','CenterName','Submitter',
                                         'SampleName','LibraryName','Library Name','Library Strategy', 'Library Source','Library Selection','LibraryLayout',
                                         'TotalReads','UniqueMappedRatio','bases','spots','spots_with_mates','avgLength','size_MB', 'Total Size, Mb', 'Total RUNs', 'Total Spots','Total Bases',
                                         'InsertSize','InsertDev','Platform','Instrument',
                                         'Study Title','Sample Title','Project Tittle','Project Description','Project Biblio',
                                         'Jose_1', 'Jose_2', 'Jose_3', 'Jose_4', 'Jose_5','Jose_6', 'Jose_7', 'Jose_8',
                                         'download_path'])

sra_without_ordenada.rename(columns={"Sample Title": "SampleTitle"},inplace = True)
sra_without_ordenada.rename(columns={"Study Title": "StudyTitle"},inplace = True)
sra_without_ordenada.rename(columns={"Project Tittle": "ProjectTittle"},inplace = True)

# sra_without_ordenada.to_csv(f'{path_output_tables}Sra(-Merge_final)Filtered_ordenada.csv', index=False)
# ------------------------------------------------------------------------------------------------------------------------------

## FILTER ABIOTIC

abiotic = {'No':[],'Yes':[],'Evaluate':[]}
countsTerms = {}
search_words = r"drought|submergence|salt|cold|X-ray|temperature|heat|wounding|\sUV[\s\.]|radiation|metal|CO2|dry|dehydrat\w+|flood\w+|\sNaCl\s\.|magnetic field|osmotic|water stress|salt+H2O2|H2O2|hypoxia|ozone|stress response|photorespiratory stress|oxidative|injury|carbon starvation|space\s?flight|cell wall damage|Gamma-IR|nanoplastics|low water"
for row in sra_without_ordenada.itertuples():
    pattern=','.join([str(row.StudyTitle),str(row.ProjectTittle)])
    if re.search(search_words, pattern, re.I):
        regular = re.search(search_words, pattern, re.I)
        
        g = countsTerms.get(regular.group(0), 0)
        countsTerms[regular.group(0)] = g+1
        
        abiotic['Yes'].append(row.Run)
    elif len(pattern) == 0:
        abiotic['Evaluate'].append(row.Run)
    else:
        abiotic['No'].append(row.Run)
        
SRA_ABIOTICS = abiotic['Yes']
dfABIOTIC = sra_without_ordenada[sra_without_ordenada['Run'].isin(SRA_ABIOTICS)]

dfABIOTIC.to_excel(f'{path_output_tables}dfsra_ABIOTIC.xlsx',index=False)


In [24]:
duplicates = dfABIOTIC[dfABIOTIC.duplicated(subset=['Run'], keep=False)]
display(duplicates)

,Submission,Study Accession,Experiment,Sample,Run,BioProject,ProjectID,BioSample,Organism Name,SampleType,...,Project Biblio,Jose_1,Jose_2,Jose_3,Jose_4,Jose_5,Jose_6,Jose_7,Jose_8,download_path
